# <center>まとめ</center>

In [ ]:
# 使用するライブラリの呼び出し
import os
import cv2
import pyocr
import pyocr.builders
import numpy as np
import pandas as pd
import pytesseract
from PIL import Image, ImageDraw
import time
import torch
import torchvision
from torchvision import models as models
from torchvision import transforms as transforms
from tqdm import tqdm

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print(torch.cuda.device_count())

0


In [ ]:
"""
使用するものを諸々呼び出しておく
"""
# インストール済みのTesseractのパスを通す
path_tesseract = "C:\\Program Files\\Tesseract-OCR"
if path_tesseract not in os.environ["PATH"].split(os.pathsep):
    os.environ["PATH"] += os.pathsep + path_tesseract

# 使用するデバイスの指定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデルの呼び出し
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)
model.eval()

In [3]:
#detframe = cv2.bitwise_not(bullet_num)
"""
1フレームから数字を取得するための関数
"""
def get_num(frame):
    bgr2g = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    bgr2g = cv2.bitwise_not(bgr2g)
    to_PIL = Image.fromarray(bgr2g)
    tools = pyocr.get_available_tools()
    if len(tools) == 0:
        print("No OCR tool")
    
    tool = tools[0]
    lang = 'eng'

    text = tool.image_to_string(
    Image.fromarray(bgr2g),
    lang=lang,
    #builder=pyocr.builders.DigitBuilder()
    builder=pyocr.builders.TextBuilder(tesseract_layout=6)
    )
    
    if text.isdigit():
        text = int(text)
    else:
        text = '-'
    
    return text
"""
画像から得られる情報(数字、フレーム誤差)をまとめてリストで作成する関数
"""
def make_data(read_name):
    # データを格納するための配列
    battle_data = []
    check_time = time.time()
    
    
    # 認識範囲
    xmin,xmax = 1753,1783 
    ymin,ymax = 960,1000
    """
    # 認識範囲
    xmin,xmax = 1143,1190 
    ymin,ymax = 640,666
    """
    read_file = cv2.VideoCapture(read_name)

    # フレーム数のカウント
    frame_cnt = 0
    frame_num = read_file.get(cv2.CAP_PROP_FRAME_COUNT)

    # 真偽値
    tf = 0
    
    # errorの初期化
    error = 0
    
    pbar = tqdm(total=frame_num)
    while True:
        ret, frame = read_file.read()
        # 初期化 1次元の[数字,フレーム数,真偽値]
        num = []
        if not ret:
            break
            
        # 画像の大きさを変更する
        #frame = cv2.resize(frame,dsize=(1280,720))
        
        # 弾数が減っている部分を切り出している
        bullet_num = frame[ymin:ymax,xmin:xmax]

        detframe = cv2.bitwise_not(bullet_num)

        # 関数get_numを呼び出して数字をカウントしていく
        number = get_num(detframe)

        # 誤差についての計算
        if frame_cnt != 0:
            # エラーの算出
            error = detframe.astype(int) - before_frame.astype(int)
            error = error.max()
        
        # 一次元配列に格納
        num.append(number)    # 数字
        num.append(frame_cnt)    # フレーム数
        num.append(tf)    # 真偽値
        num.append(error)    # フレーム誤差

        # 二次元配列に格納
        battle_data.append(num)

        # フレーム数を増やす
        frame_cnt+=1
        pbar.update(1)
        
        # ひとつ前のフレームを保存しておく
        before_frame = detframe

    read_file.release()
    last_time = time.time()
    pbar.close()
    
    print(last_time-check_time)
    return battle_data

"""
得られたデータから射撃されているかどうかを判定するアルゴリズムの関数
"""
def battle_cut(db):    # db=[num,frame_num,tf]
    # frame数,現在のフレームをカウントする変数
    frame_len = len(db)
    frame_cnt = 0
    battle_true = 1    # battleしていると判断したものはtfを1に変換する
    keep_num = 0    # 数字を保持しておくための変数
    """
    # フラットライン
    rainbow_cnt = 15    # カットするためのフレーム数の定義７フレーム動いていないと撃っていないと判断する
    start_late = 7   # 打ち始めを考慮したフレーム数
    """
    rainbow_cnt = 100
    start_late = 20
    #conti_cnt=1    # 同じ数字が何回続いたかをカウントする関数
    # 判定結果を保持するための配列(仮)
    result = []
    # battle判定
    while True:
        if frame_cnt >= frame_len:
            break
        keep_num = db[frame_cnt][0]        # 現在のフレームの時の弾数を格納しておく
        start_point = 0    # 打ち始めるポイント
        conti_cnt=0
        if db[frame_cnt][0] != "-" and 100 <= db[frame_cnt][3]:        # 数字を検知していないところはカット
            while True:    # 同じ数字が何回続いたかをカウントする
                conti_cnt+=1
                if frame_cnt+conti_cnt >=frame_len:
                    break
                if keep_num != db[frame_cnt+conti_cnt][0]:
                    break
            # print(conti_cnt)
            if conti_cnt <= rainbow_cnt:
                start_point = frame_cnt-start_late
                for i in range(conti_cnt+start_late):
                    if frame_cnt+i >=frame_len:
                        break
                    db[start_point+i][2] = int(1)
                frame_cnt+=conti_cnt
            else:
                for i in range(rainbow_cnt):
                    if frame_cnt+i >=frame_len:
                        break
                    db[frame_cnt+i][2] = int(1)
                frame_cnt+=conti_cnt
        else:
            frame_cnt+=1
    return db

"""
画面の中心を切り取る関数
"""
# 中心を切り取る関数
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))


"""
人がいるところを拡張表示する関数
"""
def human_marker(f):
    # フレームを読み込んでRGBに変換する
    #bgr2rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    
    # もっとも真ん中に近いものを表示
    wide = 0
    high = 0
    min_distant = 800
    just_high = 0
    just_wide = 0
    fin_list = []
    a = np.zeros(4)
    
    image = Image.fromarray(f)
    
    # 中心部分に着目する
    image = crop_center(image, 800,600)
    
    # pytorchで使用できるようにtensor型に変換する
    image_tensor = transforms.functional.to_tensor(image).to(device)
    
    # 推論
    out = model([image_tensor])[0]

    # 人である確率が0.4であるという条件でボックスを取り出す
    ID = 1
    boxes = out["boxes"][out["labels"] == ID & out["scores"] >= 0.4].detach().cpu().numpy()

    draw = ImageDraw.Draw(image, mode="RGBA")

    for box in boxes:
        # 四角形の中心の座標を取得する
        high = (box[0] + box[1]) / 2 
        wide = (box[2] + box[3]) / 2
        
        # 中心との距離を計算する
        distant = ((high-300)**2+(wide-400)**2)**0.5
        if min_distant > distant:
            a = box
            just_high = high
            just_wide = wide
            min_distant = distant
        draw.rectangle(box, outline="blue", width=3)    
    
    # 画像をnumpyarrayに戻す
    make_frame = np.array(image)
    
    fin_list.append(make_frame)
    fin_list.append(just_high)
    fin_list.append(just_wide)
    
    return fin_list

"""
動画を保存するための関数
"""
def capture(read_name,cut_data,write_name,point_file_name):
    read = cv2.VideoCapture(read_name)    # 動画の読み込み
    read_width = int(read.get(cv2.CAP_PROP_FRAME_WIDTH))
    read_height = int(read.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = read.get(cv2.CAP_PROP_FPS)# *0.5
    high_list = []
    wide_list = []
    
    success = "完了しました" 
    fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    write = cv2.VideoWriter(write_name,fourcc,fps,(800,600))
    
    for i in tqdm(cut_data):
        ret,frame = read.read()
        if ret:
            if i[2] == 1:
                # 人物情報の取得
                anadata = human_marker(frame)
                frame = anadata[0] 
                high = anadata[1]
                wide = anadata[2]
                # 座標取得
                high_list.append(high)
                wide_list.append(wide)
                # 描写
                write.write(frame)
    write.release()
    read.release()
    
    # データの保存
    df = pd.DataFrame({"high":high_list,
                       "wide":wide_list}, index=None)
    df.to_csv(point_file_name)
    
    return success

In [ ]:
f = "rensyu.mp4"
f1 = "cut.mp4"
f2 = "aim_plot.csv"

data = make_data(f)
edit = battle_cut(data)
capture(f,edit,f1,f2)

In [ ]:
capture(f,edit,f1,f2)

In [9]:
for i in edit:
    if i[2] == 1:
        print(i)

['-', 5891, 1, 167]
['-', 5892, 1, 184]
['-', 5893, 1, 186]
['-', 5894, 1, 152]
['-', 5895, 1, 35]
['-', 5896, 1, 2]
['-', 5897, 1, 2]
['-', 5898, 1, 2]
['-', 5899, 1, 2]
['-', 5900, 1, 4]
['-', 5901, 1, 3]
['-', 5902, 1, 3]
['-', 5903, 1, 3]
['-', 5904, 1, 3]
['-', 5905, 1, 3]
['-', 5906, 1, 4]
['-', 5907, 1, 3]
['-', 5908, 1, 3]
['-', 5909, 1, 5]
['-', 5910, 1, 12]
[7, 5911, 1, 220]
[4, 5912, 1, 250]
['-', 5988, 1, 2]
['-', 5989, 1, 2]
['-', 5990, 1, 2]
['-', 5991, 1, 2]
['-', 5992, 1, 2]
['-', 5993, 1, 0]
['-', 5994, 1, 2]
['-', 5995, 1, 3]
['-', 5996, 1, 3]
['-', 5997, 1, 2]
['-', 5998, 1, 2]
['-', 5999, 1, 0]
['-', 6000, 1, 0]
['-', 6001, 1, 0]
['-', 6002, 1, 0]
['-', 6003, 1, 0]
['-', 6004, 1, 0]
['-', 6005, 1, 1]
['-', 6006, 1, 2]
['-', 6007, 1, 32]
[7, 6008, 1, 100]
['-', 6135, 1, 0]
['-', 6136, 1, 0]
['-', 6137, 1, 0]
['-', 6138, 1, 4]
['-', 6139, 1, 0]
['-', 6140, 1, 0]
['-', 6141, 1, 1]
['-', 6142, 1, 1]
['-', 6143, 1, 1]
['-', 6144, 1, 3]
['-', 6145, 1, 3]
['-', 6146, 1, 2]

[2, 59600, 1, 3]
[2, 59601, 1, 5]
[2, 59602, 1, 4]
[2, 59603, 1, 5]
[2, 59604, 1, 5]
[2, 59605, 1, 5]
[2, 59606, 1, 8]
[2, 59607, 1, 7]
[2, 59608, 1, 8]
[2, 59609, 1, 4]
[2, 59610, 1, 15]
[2, 59611, 1, 41]
[2, 59612, 1, 20]
[2, 59613, 1, 12]
[2, 59614, 1, 9]
[2, 59615, 1, 7]
[2, 59616, 1, 7]
[2, 59617, 1, 4]
[2, 59618, 1, 3]
[2, 59619, 1, 5]
[2, 59620, 1, 5]
[2, 59621, 1, 4]
[2, 59622, 1, 5]
[2, 59623, 1, 3]
[2, 59624, 1, 3]
[2, 59625, 1, 2]
[2, 59626, 1, 3]
[2, 59627, 1, 7]
[2, 59628, 1, 15]
[2, 59629, 1, 11]
[2, 59630, 1, 15]
[2, 59631, 1, 1]
[2, 59632, 1, 1]
[2, 59633, 1, 2]
[2, 59634, 1, 3]
[2, 59635, 1, 78]
[2, 59636, 1, 104]
[2, 59637, 1, 20]
[2, 59638, 1, 15]
[2, 59639, 1, 17]
[2, 59640, 1, 2]
[2, 59641, 1, 2]
[2, 59642, 1, 2]
[2, 59643, 1, 3]
[2, 59644, 1, 2]
[2, 59645, 1, 3]
[2, 59646, 1, 17]
[2, 59647, 1, 18]
[2, 59648, 1, 21]
[2, 59649, 1, 7]
[2, 59650, 1, 9]
[2, 59651, 1, 1]
[2, 59652, 1, 2]
[2, 59653, 1, 15]
[2, 59654, 1, 1]
[2, 59655, 1, 2]
[2, 59656, 1, 3]
[2, 59657, 1, 

[2, 97306, 1, 55]
[2, 97307, 1, 6]
[2, 97308, 1, 7]
[2, 97309, 1, 7]
[2, 97310, 1, 8]
[2, 97311, 1, 11]
[2, 97312, 1, 14]
[2, 97313, 1, 51]
[2, 97314, 1, 15]
[2, 97315, 1, 37]
[2, 97316, 1, 15]
[2, 97317, 1, 20]
[2, 97318, 1, 16]
[2, 97319, 1, 9]
[2, 97320, 1, 14]
[2, 97321, 1, 12]
[2, 97322, 1, 15]
[2, 97323, 1, 17]
[2, 97324, 1, 41]
[2, 97325, 1, 14]
[2, 97326, 1, 39]
[2, 97327, 1, 13]
[2, 97328, 1, 24]
[2, 97329, 1, 19]
[2, 97330, 1, 11]
[2, 97331, 1, 21]
[2, 97332, 1, 11]
[2, 97333, 1, 5]
[1, 97334, 1, 195]
[1, 97335, 1, 12]
[1, 97336, 1, 4]
[1, 97337, 1, 4]
[1, 97338, 1, 2]
[1, 97339, 1, 2]
[1, 97340, 1, 3]
[1, 97341, 1, 10]
[1, 97342, 1, 6]
[1, 97343, 1, 4]
[1, 97344, 1, 11]
[1, 97345, 1, 13]
[1, 97346, 1, 7]
[1, 97347, 1, 7]
[1, 97348, 1, 7]
[1, 97349, 1, 9]
[1, 97350, 1, 19]
[1, 97351, 1, 0]
[1, 97352, 1, 3]
[1, 97353, 1, 2]
[1, 97354, 1, 2]
[1, 97355, 1, 4]
[1, 97356, 1, 2]
[1, 97357, 1, 6]
[1, 97358, 1, 2]
[1, 97359, 1, 3]
[1, 97360, 1, 4]
[1, 97361, 1, 4]
[1, 97362, 1, 11]
[

[8, 134873, 1, 1]
['-', 134874, 1, 19]
['-', 134875, 1, 0]
['-', 134876, 1, 0]
['-', 134877, 1, 0]
['-', 134878, 1, 0]
['-', 134879, 1, 0]
[8, 134880, 1, 3]
[8, 134881, 1, 0]
[8, 134882, 1, 1]
[8, 134883, 1, 0]
[8, 134884, 1, 0]
[7, 134885, 1, 233]
[7, 134886, 1, 8]
[7, 134887, 1, 3]
[7, 134888, 1, 11]
[7, 134889, 1, 3]
[7, 134890, 1, 5]
[7, 134891, 1, 9]
[7, 134892, 1, 8]
[7, 134893, 1, 3]
[7, 134894, 1, 4]
[7, 134895, 1, 7]
['-', 135241, 1, 5]
['-', 135242, 1, 4]
['-', 135243, 1, 5]
['-', 135244, 1, 5]
['-', 135245, 1, 2]
['-', 135246, 1, 7]
['-', 135247, 1, 1]
['-', 135248, 1, 2]
['-', 135249, 1, 5]
['-', 135250, 1, 1]
['-', 135251, 1, 4]
['-', 135252, 1, 5]
['-', 135253, 1, 9]
['-', 135254, 1, 5]
['-', 135255, 1, 4]
['-', 135256, 1, 2]
['-', 135257, 1, 8]
['-', 135258, 1, 5]
['-', 135259, 1, 0]
['-', 135260, 1, 8]
[5, 135261, 1, 191]
[5, 135262, 1, 15]
[5, 135263, 1, 7]
[5, 135264, 1, 3]
[5, 135265, 1, 3]
[5, 135266, 1, 4]
[5, 135267, 1, 7]
[5, 135268, 1, 3]
['-', 136596, 1, 2]
['-

In [ ]:
# データの確認
df = pd.read_csv(f2)